In [1]:
import pubmed_parser as pp
import pandas as pd
import nltk
nltk.download('punkt')
import numpy as np
from csv import DictWriter
import json
import math

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

#Source: https://stackoverflow.com/questions/12118720/python-tf-idf-cosine-to-find-document-similarity

[nltk_data] Downloading package punkt to /home/gitpod/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
path_xml = pp.list_xml_path('../../data/raw') # list all xml paths under directory
pubmed_dict = pp.parse_medline_xml(path_xml[0]) # dictionary output

In [3]:
pubmed_dict

[{'title': 'Ca2+-Ca2+ exchange catalyzed by the membrane-bound Ca2+, Mg2+-ATPase of sarcoplasmic reticulum vesicles.',
  'issue': '24()',
  'pages': '423-34',
  'abstract': 'Sarcoplasmic reticulum vesicles were preloaded with either 45Ca2+ or unlabeled Ca2+ X 45Ca2+ efflux and influx, together with phosphorylation of the membrane-bound Ca2+, Mg2+-ATPase, were determined in the presence of either ATP and ADP or acetyl phosphate. ATP induced 45Ca2+ efflux. This ATP-induced 45Ca2+ efflux depended on ADP, external Ca2+, and Mg2+. The Ca2+ concentration dependence of the efflux was quite similar to the Ca2+ concentration dependence of the ATP-induced 45Ca2+ influx and the enzyme phosphorylation. The rate of the efflux was proportional to the steady level of the phosphoenzyme. The affinity for free ADP in this efflux was extremely high, being in good agreement with the affinity for free ADP in the transphosphorylation from the phosphoenzyme to ADP. These results show that the ATP-induced 45C

In [4]:
abstracts = [_['abstract'] for _ in pubmed_dict]
nlm_ids = [_['nlm_unique_id'] for _ in pubmed_dict]

In [27]:
vectorizer = TfidfVectorizer()

In [28]:
abstracts_tfidf = vectorizer.fit_transform(abstracts)

In [29]:
def get_consine_similarity_tf_idf(vectorizer,abstracts_tfidf, query):
    query_tfidf = vectorizer.transform([query])
    cosine_similarities = cosine_similarity(query_tfidf, abstracts_tfidf).flatten()
    return cosine_similarities

In [33]:
def get_top_n_cosine_similarity_tf_idf_docs(vectorizer, abstracts_tfidf, query, n=10):
    cosine_similarities = get_consine_similarity_tf_idf(vectorizer,abstracts_tfidf, query)
    related_docs_indices = cosine_similarities.argsort()[:-n - 1:-1]
    return [(index, cosine_similarities[index]) for index in related_docs_indices]

In [40]:
def get_top_n_nlm_ids_cosine_similarity_tf_idf_docs(vectorizer, abstracts_tfidf, query, n=10):
    related_docs_indices = get_top_n_cosine_similarity_tf_idf_docs(vectorizer, abstracts_tfidf, query, n)
    return [(nlm_ids[index], score) for index, score in related_docs_indices]

In [48]:
def get_full_nlm_docs_cosine_similarity_tf_idf_docs(vectorizer, abstracts_tfidf, query, n=10):
    related_docs_indices = get_top_n_cosine_similarity_tf_idf_docs(vectorizer, abstracts_tfidf, query, n)
    return [(pubmed_dict[index], score) for index, score in related_docs_indices]

In [49]:
get_full_nlm_docs_cosine_similarity_tf_idf_docs(vectorizer, abstracts_tfidf, 'phosphoenzyme')

[({'title': 'Ca2+-Ca2+ exchange catalyzed by the membrane-bound Ca2+, Mg2+-ATPase of sarcoplasmic reticulum vesicles.',
   'issue': '24()',
   'pages': '423-34',
   'abstract': 'Sarcoplasmic reticulum vesicles were preloaded with either 45Ca2+ or unlabeled Ca2+ X 45Ca2+ efflux and influx, together with phosphorylation of the membrane-bound Ca2+, Mg2+-ATPase, were determined in the presence of either ATP and ADP or acetyl phosphate. ATP induced 45Ca2+ efflux. This ATP-induced 45Ca2+ efflux depended on ADP, external Ca2+, and Mg2+. The Ca2+ concentration dependence of the efflux was quite similar to the Ca2+ concentration dependence of the ATP-induced 45Ca2+ influx and the enzyme phosphorylation. The rate of the efflux was proportional to the steady level of the phosphoenzyme. The affinity for free ADP in this efflux was extremely high, being in good agreement with the affinity for free ADP in the transphosphorylation from the phosphoenzyme to ADP. These results show that the ATP-induced